In [ ]:
!pip install pyspark

In [ ]:
!pip install pyarrow ## pandas dataframe

In [3]:
# Import libraries

from pyspark.sql import SparkSession

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import pyspark.sql.functions as fn
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer
from pyspark.sql.functions import udf, StringType

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Initializing Spark Session Object
spark = SparkSession.builder.master('local').getOrCreate()

In [5]:
path = '/content/drive/MyDrive/Marketing-Analytics/Data/Telco_Churn_Data.csv'

In [6]:
df1=spark.read.csv(path,inferSchema=True, header=True)

In [7]:
df1.show(6)

+------------+-----------+----------------+----------+------------------+-----------+--------------------+-----------------+---------------+-------------------+-------------------------+---------------------------+-------------+----------------------------+--------------------------+
|Target Churn|Target Code|Current Bill Amt| Avg Calls|Avg Calls Weekdays|Account Age|Percent Increase MOM|Acct Plan Subtype| Complaint Code|Avg Days Delinquent|Current TechSupComplaints|Current Days OpenWorkOrders|Equipment Age|Condition of Current Handset|Avg Hours WorkOrderOpenned|
+------------+-----------+----------------+----------+------------------+-----------+--------------------+-----------------+---------------+-------------------+-------------------------+---------------------------+-------------+----------------------------+--------------------------+
|    No Churn|          0|          184.73|   17950.0|           30297.0|         24|          -0.3341935|             Gold|Billing Problem|     

In [8]:
data = df1.withColumnRenamed('Avg Hours WorkOrderOpenned', 'Avg Hours WorkOrderOpened') #rename columns: original name, new name

In [9]:
# check what percentage of missing observations are there in each column
data.agg(*[
    (1 - (fn.count(c) / fn.count('*'))).alias(c + '_missing')
    for c in data.columns
]).show()

+--------------------+-------------------+------------------------+-----------------+--------------------------+-------------------+----------------------------+-------------------------+----------------------+---------------------------+---------------------------------+-----------------------------------+---------------------+------------------------------------+---------------------------------+
|Target Churn_missing|Target Code_missing|Current Bill Amt_missing|Avg Calls_missing|Avg Calls Weekdays_missing|Account Age_missing|Percent Increase MOM_missing|Acct Plan Subtype_missing|Complaint Code_missing|Avg Days Delinquent_missing|Current TechSupComplaints_missing|Current Days OpenWorkOrders_missing|Equipment Age_missing|Condition of Current Handset_missing|Avg Hours WorkOrderOpened_missing|
+--------------------+-------------------+------------------------+-----------------+--------------------------+-------------------+----------------------------+-------------------------+---------

In [10]:
data.describe().show()

+-------+------------+-------------------+------------------+------------------+------------------+-----------------+--------------------+-----------------+---------------+-------------------+-------------------------+---------------------------+------------------+----------------------------+-------------------------+
|summary|Target Churn|        Target Code|  Current Bill Amt|         Avg Calls|Avg Calls Weekdays|      Account Age|Percent Increase MOM|Acct Plan Subtype| Complaint Code|Avg Days Delinquent|Current TechSupComplaints|Current Days OpenWorkOrders|     Equipment Age|Condition of Current Handset|Avg Hours WorkOrderOpened|
+-------+------------+-------------------+------------------+------------------+------------------+-----------------+--------------------+-----------------+---------------+-------------------+-------------------------+---------------------------+------------------+----------------------------+-------------------------+
|  count|        4708|               

In [11]:
data.groupBy("Complaint Code").count().orderBy("count", ascending=False).first()[0]

'Billing Problem'

In [12]:
data.groupBy("Condition of Current Handset").count().orderBy("count", ascending=False).first()[0]

1

In [13]:
# impute with mode 
df2 = data.fillna({'Complaint Code':'Billing Problem', 'Condition of Current Handset': 1})

In [14]:
# check what percentage of missing observations are there in each column
df2.agg(*[
    (1 - (fn.count(c) / fn.count('*'))).alias(c + '_missing')
    for c in df2.columns
]).show()

+--------------------+-------------------+------------------------+-----------------+--------------------------+-------------------+----------------------------+-------------------------+----------------------+---------------------------+---------------------------------+-----------------------------------+---------------------+------------------------------------+---------------------------------+
|Target Churn_missing|Target Code_missing|Current Bill Amt_missing|Avg Calls_missing|Avg Calls Weekdays_missing|Account Age_missing|Percent Increase MOM_missing|Acct Plan Subtype_missing|Complaint Code_missing|Avg Days Delinquent_missing|Current TechSupComplaints_missing|Current Days OpenWorkOrders_missing|Equipment Age_missing|Condition of Current Handset_missing|Avg Hours WorkOrderOpened_missing|
+--------------------+-------------------+------------------------+-----------------+--------------------------+-------------------+----------------------------+-------------------------+---------

In [15]:
df2.describe().show()

+-------+------------+-------------------+------------------+------------------+------------------+-----------------+--------------------+-----------------+---------------+-------------------+-------------------------+---------------------------+------------------+----------------------------+-------------------------+
|summary|Target Churn|        Target Code|  Current Bill Amt|         Avg Calls|Avg Calls Weekdays|      Account Age|Percent Increase MOM|Acct Plan Subtype| Complaint Code|Avg Days Delinquent|Current TechSupComplaints|Current Days OpenWorkOrders|     Equipment Age|Condition of Current Handset|Avg Hours WorkOrderOpened|
+-------+------------+-------------------+------------------+------------------+------------------+-----------------+--------------------+-----------------+---------------+-------------------+-------------------------+---------------------------+------------------+----------------------------+-------------------------+
|  count|        4708|               

In [16]:
df2.printSchema()

root
 |-- Target Churn: string (nullable = true)
 |-- Target Code: integer (nullable = true)
 |-- Current Bill Amt: double (nullable = true)
 |-- Avg Calls: double (nullable = true)
 |-- Avg Calls Weekdays: double (nullable = true)
 |-- Account Age: integer (nullable = true)
 |-- Percent Increase MOM: double (nullable = true)
 |-- Acct Plan Subtype: string (nullable = true)
 |-- Complaint Code: string (nullable = false)
 |-- Avg Days Delinquent: double (nullable = true)
 |-- Current TechSupComplaints: integer (nullable = true)
 |-- Current Days OpenWorkOrders: double (nullable = true)
 |-- Equipment Age: integer (nullable = true)
 |-- Condition of Current Handset: integer (nullable = false)
 |-- Avg Hours WorkOrderOpened: double (nullable = true)



In [17]:
def udf_Complaint_Code(Complaint): ## name of the UDF
      if (Complaint == 'Billing Problem'):
        return 0
      elif (Complaint == 'Call Quality'):
        return 1
      elif (Complaint == 'Check Account'):
        return 2
      elif (Complaint == 'Inaccurate Sales Inf'):
        return 3
      elif (Complaint == 'Moving'):
        return 4
      else: return 5

In [18]:
complCode_udf = udf(udf_Complaint_Code)
df3=df2.withColumn("Complaint Code", complCode_udf('Complaint Code'))

In [19]:
from typing import cast
df4 = df3.withColumn('Complaint Code', df3['Complaint Code'].cast('integer'))

In [20]:
df4.select(['Complaint Code']).distinct().show()

+--------------+
|Complaint Code|
+--------------+
|             1|
|             3|
|             5|
|             4|
|             2|
|             0|
+--------------+



In [21]:
df4.printSchema()

root
 |-- Target Churn: string (nullable = true)
 |-- Target Code: integer (nullable = true)
 |-- Current Bill Amt: double (nullable = true)
 |-- Avg Calls: double (nullable = true)
 |-- Avg Calls Weekdays: double (nullable = true)
 |-- Account Age: integer (nullable = true)
 |-- Percent Increase MOM: double (nullable = true)
 |-- Acct Plan Subtype: string (nullable = true)
 |-- Complaint Code: integer (nullable = true)
 |-- Avg Days Delinquent: double (nullable = true)
 |-- Current TechSupComplaints: integer (nullable = true)
 |-- Current Days OpenWorkOrders: double (nullable = true)
 |-- Equipment Age: integer (nullable = true)
 |-- Condition of Current Handset: integer (nullable = false)
 |-- Avg Hours WorkOrderOpened: double (nullable = true)



In [22]:
###split data into training and testing
train_data,test_data=df4.randomSplit([0.7,0.3]) ##70% for training

In [23]:
Pred_corr= ['Avg Days Delinquent','Percent Increase MOM', 'Avg Calls Weekdays','Current Bill Amt', 'Avg Calls','Account Age']

In [24]:
# Initializing Vector Assembler to convert columns to vector
vector_col = "Predictors"
assembler = VectorAssembler(inputCols=Pred_corr, 
                            outputCol=vector_col) ##assemble the inputs and outputs

In [25]:
# Initializing Random Forest Model
classifier=RandomForestClassifier(featuresCol="Predictors",labelCol="Target Code",predictionCol="Prediction Churn")

In [26]:
# Initializing Pipeline to execute all steps at once
pipeline = Pipeline(stages=[assembler, classifier])

In [27]:
# Creating a Parameter Grid with all the parameters 
paramGrid = ParamGridBuilder() \
                  .addGrid(classifier.maxDepth, [3, 5, 10, 15]) \
                  .addGrid(classifier.numTrees, [3, 5, 10, 15]) \
                  .build()

In [28]:
# evaluate model
evaluator = MulticlassClassificationEvaluator(labelCol="Target Code", predictionCol="Prediction Churn")

In [29]:
# Initialiazing cross validator model with paramGrid to create parameterized tuned model
crossValidator = CrossValidator(estimator=pipeline,
                             estimatorParamMaps=paramGrid,
                             evaluator=evaluator,
                             numFolds=10)

In [30]:
# Training cross validator model
tuned_model = crossValidator.fit(train_data)

In [31]:
# Getting predictions
predictions = tuned_model.transform(test_data)

In [32]:
# Evaluating accuracy of the model

evaluator = MulticlassClassificationEvaluator() \
                      .setLabelCol("Target Code") \
                      .setPredictionCol("Prediction Churn") \
                      .setMetricName("accuracy")

accuracy = evaluator.evaluate(predictions)

print("Test Error : {}".format(100*(1.0 - accuracy)))
print("Accuracy of the model : {}".format(100*accuracy))

Test Error : 21.6551724137931
Accuracy of the model : 78.3448275862069
